In [9]:
import os
from dotenv import load_dotenv
from binance.client import Client

# Load the keys from the invisible .env file
load_dotenv()

api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET")

# Safety Check: Print a warning if keys are missing (Optional)
if not api_key:
    print("CRITICAL ERROR: API Key not found!")

client = Client(api_key, api_secret)

In [10]:
import requests
import pandas as pd

url = "https://api.coingecko.com/api/v3/coins/markets"

params = {
    "vs_currency": "usd",
    "order": "market_cap_desc",
    "per_page": 50,   # pull more so filtering still leaves ~20
    "page": 1,
    "sparkline": False
}

response = requests.get(url, params=params)
data = response.json()

df = pd.DataFrame(data)[
    ["market_cap_rank", "symbol", "name", "market_cap", "current_price"]
]

# -----------------------------
# Filters
# -----------------------------

stablecoins = {
    "usdt", "usdc", "busd", "dai", "tusd",
    "usdp", "fdusd", "gusd", "lusd",
    "ust", "ustc", "usdd", "eurc",'usds','usde'
}

# normalize text
df["symbol"] = df["symbol"].str.lower()
df["name"] = df["name"].str.lower()

# remove stablecoins
df = df[~df["symbol"].isin(stablecoins)]

# remove wrapped coins
df = df[
    ~df["name"].str.contains("wrapped") &
    ~df["symbol"].str.startswith("w")
]

# take top 20 after filtering
df = df.sort_values("market_cap", ascending=False).head(20)

print(df)

    market_cap_rank      symbol                                    name  \
0                 1         btc                                 bitcoin   
1                 2         eth                                ethereum   
3                 4         bnb                                     bnb   
4                 5         xrp                                     xrp   
5                 6         sol                                  solana   
7                 8       steth                       lido staked ether   
8                 9         trx                                    tron   
9                10        doge                                dogecoin   
10               11  figr_heloc                            figure heloc   
11               12         ada                                 cardano   
13               14         bch                            bitcoin cash   
19               20     bsc-usd  binance bridged usdt (bnb smart chain)   
20               21      